In [5]:
import pandas as pd
import numpy as np
import psycopg2
from sql_functions import get_dataframe
from sql_functions import get_engine
engine = get_engine()


In [6]:
etfs_lowrisk_sql = 'SELECT * FROM public.etfs_lowrisk_test'
etfs_lowrisk = get_dataframe(etfs_lowrisk_sql)

In [7]:
etfs_lowrisk.head()

,ticker,fund_name,one_yr_return,expense_ratio,aum,analyst_pick,grade,efficiency,tradability,fit,...,quality_score_10,score_global_rank,sri_screening_criteria_exposure_%,carbon_intensity,main_segment,sub_segment,annual_to_monthly,roi,roi_7,roi_12
0,TEGS,Trend Aggregation ESG ETF,NaN,1.43,3.645400e+05,True,NaN,48.0,NaN,NaN,...,NaN,NaN,NaN,NaN,Asset Allocation,Global - Target Outcome,NaN,NaN,NaN,None
1,IWN,iShares Russell 2000 Value ETF,5.09,0.23,1.192000e+10,True,A,96.0,98.0,85.0,...,5.00,24.44,3.44,189.85,Equity,U.S. - Small Cap Value,0.005544,113971.278929,10661.728057,None
2,PAVE,Global X U.S. Infrastructure Development ETF,32.39,0.47,5.000000e+09,True,B,87.0,97.0,NaN,...,6.63,56.45,NaN,226.69,Equity,U.S. Infrastructure,NaN,NaN,NaN,None
3,FLOT,iShares Floating Rate Bond ETF,5.97,0.15,7.140000e+09,True,A,94.0,100.0,NaN,...,7.45,86.56,3.72,63.23,Fixed Income,"U.S. - Broad Market, Broad-based Investment G...",0.001283,45716.099348,8863.197425,None
4,IBDQ,iShares iBonds Dec 2025 Term Corporate ETF,1.11,0.10,2.090000e+09,True,A,96.0,99.0,NaN,...,6.77,62.25,8.25,187.22,Fixed Income,"U.S. - Corporate, Broad-based Investment Grad...",NaN,NaN,NaN,None


In [13]:
##pip install quandl

In [14]:
#import quandl as quandl

In [15]:
from dotenv import load_dotenv
load_dotenv()
import os

In [16]:
#quandl.ApiConfig.api_key = os.getenv('quandl.ApiConfig.api_key')

In [17]:
#pip install yfinance

In [18]:
#pip install yahoo_fin

In [19]:
pip install yahooquery

Note: you may need to restart the kernel to use updated packages.


In [20]:
import yahooquery as yq

In [21]:
import yfinance as yf

In [22]:
y_ticker = etfs_lowrisk["ticker"]
print(y_ticker.info)

<bound method Series.info of 0       TEGS
1        IWN
2       PAVE
3       FLOT
4       IBDQ
        ... 
2072    GRZZ
2073    FLZA
2074     EKG
2075    ODDS
2076    RESI
Name: ticker, Length: 2077, dtype: object>


In [23]:
from yahooquery import Ticker

In [24]:
import requests
import requests_futures

In [25]:
from yahooquery import Screener
s = Screener()

In [26]:
etf_tickers = etfs_lowrisk['ticker']
y_etf_tickers = Ticker(etf_tickers.head(2))
y_assets = y_etf_tickers.asset_profile

In [27]:
import json

In [28]:
print(type(y_assets))

<class 'dict'>


In [29]:
for k, v in y_assets.items():
    print(k + ':', v)

TEGS: {'phone': '866-904-0406', 'longBusinessSummary': 'The fund will under normal conditions invest at least 80% of its net assets (plus borrowings for investment purposes) directly, or indirectly through mutual funds or exchange traded funds, in equity or fixed income securities of well-known environmental, social, and governance ("ESG") indices such as MSCI and Russell. It may also invest up to 20% of its net assets in cash and cash equivalents including U.S. government securities.', 'companyOfficers': [], 'maxAge': 86400}
IWN: {'phone': '1-800-474-2737', 'longBusinessSummary': 'The fund generally invests at least 80% of its assets in the component securities of its underlying index and in investments that have economic characteristics that are substantially identical to the component securities of its underlying index (i.e., depositary receipts representing securities of the underlying index) and may invest up to 20% of its assets in certain futures, options and swap contracts, cas

In [30]:
df_assets = pd.DataFrame.from_dict(y_assets, orient = 'index', 
                       columns=['ticker', 'phone', 'longBusinessSummary', 'companyOfficers', 'maxAge'])


In [31]:
df_assets

,ticker,phone,longBusinessSummary,companyOfficers,maxAge
TEGS,NaN,866-904-0406,The fund will under normal conditions invest a...,[],86400
IWN,NaN,1-800-474-2737,The fund generally invests at least 80% of its...,[],86400


In [32]:
df_assets.drop('ticker', axis = 1, inplace=True)

In [33]:
df_assets

,phone,longBusinessSummary,companyOfficers,maxAge
TEGS,866-904-0406,The fund will under normal conditions invest a...,[],86400
IWN,1-800-474-2737,The fund generally invests at least 80% of its...,[],86400


In [34]:
df_assets.reset_index(inplace=True)
df_assets

,index,phone,longBusinessSummary,companyOfficers,maxAge
0,TEGS,866-904-0406,The fund will under normal conditions invest a...,[],86400
1,IWN,1-800-474-2737,The fund generally invests at least 80% of its...,[],86400


In [35]:
df_assets = df_assets.rename(columns={'index':'ticker'})
df_assets

,ticker,phone,longBusinessSummary,companyOfficers,maxAge
0,TEGS,866-904-0406,The fund will under normal conditions invest a...,[],86400
1,IWN,1-800-474-2737,The fund generally invests at least 80% of its...,[],86400


In [36]:

# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        etfs.to_sql(name=table_name, # Name of SQL table variable
                        con=engine, # Engine or connection
                        schema='public', # your class schema variable
                        if_exists='replace', # Drop the table before inserting new values 
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None
else:
    print('No engine')

name 'etfs' is not defined


In [37]:
etfs_lowrisk_sql = 'SELECT * FROM public.etfs_lowrisk_test'
etfs_lowrisk = get_dataframe(etfs_lowrisk_sql)

In [38]:
etfs_lowrisk.head()

,ticker,fund_name,one_yr_return,expense_ratio,aum,analyst_pick,grade,efficiency,tradability,fit,...,quality_score_10,score_global_rank,sri_screening_criteria_exposure_%,carbon_intensity,main_segment,sub_segment,annual_to_monthly,roi,roi_7,roi_12
0,TEGS,Trend Aggregation ESG ETF,NaN,1.43,3.645400e+05,True,NaN,48.0,NaN,NaN,...,NaN,NaN,NaN,NaN,Asset Allocation,Global - Target Outcome,NaN,NaN,NaN,None
1,IWN,iShares Russell 2000 Value ETF,5.09,0.23,1.192000e+10,True,A,96.0,98.0,85.0,...,5.00,24.44,3.44,189.85,Equity,U.S. - Small Cap Value,0.005544,113971.278929,10661.728057,None
2,PAVE,Global X U.S. Infrastructure Development ETF,32.39,0.47,5.000000e+09,True,B,87.0,97.0,NaN,...,6.63,56.45,NaN,226.69,Equity,U.S. Infrastructure,NaN,NaN,NaN,None
3,FLOT,iShares Floating Rate Bond ETF,5.97,0.15,7.140000e+09,True,A,94.0,100.0,NaN,...,7.45,86.56,3.72,63.23,Fixed Income,"U.S. - Broad Market, Broad-based Investment G...",0.001283,45716.099348,8863.197425,None
4,IBDQ,iShares iBonds Dec 2025 Term Corporate ETF,1.11,0.10,2.090000e+09,True,A,96.0,99.0,NaN,...,6.77,62.25,8.25,187.22,Fixed Income,"U.S. - Corporate, Broad-based Investment Grad...",NaN,NaN,NaN,None


In [39]:
df_test = etfs_lowrisk[etfs_lowrisk['ticker'] == "SPY"]
df_test.head()

,ticker,fund_name,one_yr_return,expense_ratio,aum,analyst_pick,grade,efficiency,tradability,fit,...,quality_score_10,score_global_rank,sri_screening_criteria_exposure_%,carbon_intensity,main_segment,sub_segment,annual_to_monthly,roi,roi_7,roi_12
132,SPY,SPDR S&P 500 ETF Trust,17.03,0.09,4.259800e+11,True,A,99.0,100.0,96.0,...,6.68,58.71,9.75,122.31,Equity,U.S. - Large Cap,0.009849,335693.21783,12975.024587,None


In [40]:
yearly_amount = 1200
interest_ratio_test = 1.1248
monthly_savings = 100
expense_ratio_test = 0.09
p_ten_yr = df_test['p_ten_yr'].div(100)
interest_ratio = 1 + p_ten_yr

net = yearly_amount * interest_ratio_test - (yearly_amount * interest_ratio_test) * expense_ratio_test
print(net)
type(net)

1228.2816


float

In [41]:

net2 = yearly_amount * interest_ratio.astype(float) - (yearly_amount * interest_ratio.astype(float)) * df_test['expense_ratio'].astype(float)
print(net2)

132    1228.2816
dtype: float64


In [42]:
etfs_lowrisk['annual_interest'] = 1 + etfs_lowrisk['p_ten_yr'].astype(float) /100

In [43]:
etfs_lowrisk.head()

,ticker,fund_name,one_yr_return,expense_ratio,aum,analyst_pick,grade,efficiency,tradability,fit,...,score_global_rank,sri_screening_criteria_exposure_%,carbon_intensity,main_segment,sub_segment,annual_to_monthly,roi,roi_7,roi_12,annual_interest
0,TEGS,Trend Aggregation ESG ETF,NaN,1.43,3.645400e+05,True,NaN,48.0,NaN,NaN,...,NaN,NaN,NaN,Asset Allocation,Global - Target Outcome,NaN,NaN,NaN,None,NaN
1,IWN,iShares Russell 2000 Value ETF,5.09,0.23,1.192000e+10,True,A,96.0,98.0,85.0,...,24.44,3.44,189.85,Equity,U.S. - Small Cap Value,0.005544,113971.278929,10661.728057,None,1.0686
2,PAVE,Global X U.S. Infrastructure Development ETF,32.39,0.47,5.000000e+09,True,B,87.0,97.0,NaN,...,56.45,NaN,226.69,Equity,U.S. Infrastructure,NaN,NaN,NaN,None,NaN
3,FLOT,iShares Floating Rate Bond ETF,5.97,0.15,7.140000e+09,True,A,94.0,100.0,NaN,...,86.56,3.72,63.23,Fixed Income,"U.S. - Broad Market, Broad-based Investment G...",0.001283,45716.099348,8863.197425,None,1.0155
4,IBDQ,iShares iBonds Dec 2025 Term Corporate ETF,1.11,0.10,2.090000e+09,True,A,96.0,99.0,NaN,...,62.25,8.25,187.22,Fixed Income,"U.S. - Corporate, Broad-based Investment Grad...",NaN,NaN,NaN,None,NaN


In [44]:
years = list(range(1,61))
years

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60]

In [45]:
df_net_calc = pd.DataFrame({"year":[years], "annual_w_interest":['NaN'], "expenses":['NaN'], "net_amount":['NaN']})
df_net_calc

,year,annual_w_interest,expenses,net_amount
0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",NaN,NaN,NaN


In [46]:
df_net_calc.set_index('year')

,annual_w_interest,expenses,net_amount
year,,,
"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60]",NaN,NaN,NaN


In [47]:
monthly_savings = 100
yearly_amount = monthly_savings * 12
interest_ratio_test = 1.1248
expense_ratio_test = 0.09
#net_amount = (net_amount[i-1] + yearly_amount[i]) * interest_ratio_test - ((net_amount[i-1] + yearly_amount[i]) * interest_ratio_test) * expense_ratio_test]


df.loc[0, 'C'] = df.loc[0, 'D']

for i in range(1, len(df)):
    df.loc[i, 'C'] = df.loc[i-1, 'C'] * df.loc[i, 'A'] + df.loc[i, 'B']

In [52]:
initial_investment = 0
desired_total_savings = 142800
years = 7
etf_performance = 1.1248
expense_ratio = 0.09

In [53]:
def calculate_monthly_payment_for_savings(desired_total_savings, initial_investment, etf_performance, expense_ratio, years):
    """
    Calculate the required monthly deposit to achieve a desired total savings over a given number of years.
    
    Parameters:
        desired_total_savings (float): Desired total savings.
        initial_investment (float): Initial investment amount.
        etf_performance (float): ETF performance factor (1 + annual growth rate).
        expense_ratio (float): Annual expense ratio of the ETF (as a decimal).
        years (int): Number of years for the investment.
        
    Returns:
        float: Required monthly deposit amount.
    """
    monthly_deposit = 0.1  # Initial guess for the monthly deposit
    epsilon = 0.01  # Tolerance for the difference between actual and desired savings
    
    while True:
        total_savings = initial_investment
        for year in range(1, years + 1):
            total_savings = (monthly_deposit * 12 + total_savings) * etf_performance - (monthly_deposit * 12 + total_savings) * etf_performance * expense_ratio
        
        if abs(total_savings - desired_total_savings) < epsilon:
            break
        
        # Adjust the monthly deposit based on the difference between actual and desired savings
        monthly_deposit += (desired_total_savings - total_savings) / (12 * years * etf_performance)
    
    return monthly_deposit



In [54]:
monthly_deposit = calculate_monthly_payment_for_savings(desired_total_savings, initial_investment, etf_performance, expense_ratio, years)
print(f"Required monthly deposit to achieve ${desired_total_savings:.2f} in {years} years: ${monthly_deposit:.2f}")

Required monthly deposit to achieve $142800.00 in 7 years: $1547.07


In [2]:
import pandas as pd
import numpy as np

In [3]:
df_performance = pd.read_csv("data/p_ten_yr_imputed.csv")

In [4]:
df_performance.head()

,Unnamed: 0,ticker,simple_mean,simple_median,knn-imputer
0,0,SPY,12.48,12.48,12.48
1,1,IVV,12.54,12.54,12.54
2,2,VOO,12.55,12.55,12.55
3,3,VTI,11.98,11.98,11.98
4,4,QQQ,18.66,18.66,18.66


In [8]:
etfs_lowrisk_new = etfs_lowrisk.merge(df_performance, how='inner', on = 'ticker')

In [9]:
etfs_lowrisk_new.head()

,ticker,fund_name,one_yr_return,expense_ratio,aum,analyst_pick,grade,efficiency,tradability,fit,...,main_segment,sub_segment,annual_to_monthly,roi,roi_7,roi_12,Unnamed: 0,simple_mean,simple_median,knn-imputer
0,TEGS,Trend Aggregation ESG ETF,NaN,1.43,3.645400e+05,True,NaN,48.0,NaN,NaN,...,Asset Allocation,Global - Target Outcome,NaN,NaN,NaN,None,2179,5.3453,6.07,-26.356
1,IWN,iShares Russell 2000 Value ETF,5.09,0.23,1.192000e+10,True,A,96.0,98.0,85.0,...,Equity,U.S. - Small Cap Value,0.005544,113971.278929,10661.728057,None,114,6.8600,6.86,6.860
2,PAVE,Global X U.S. Infrastructure Development ETF,32.39,0.47,5.000000e+09,True,B,87.0,97.0,NaN,...,Equity,U.S. Infrastructure,NaN,NaN,NaN,None,226,5.3453,6.07,11.706
3,FLOT,iShares Floating Rate Bond ETF,5.97,0.15,7.140000e+09,True,A,94.0,100.0,NaN,...,Fixed Income,"U.S. - Broad Market, Broad-based Investment G...",0.001283,45716.099348,8863.197425,None,173,1.5500,1.55,1.550
4,IBDQ,iShares iBonds Dec 2025 Term Corporate ETF,1.11,0.10,2.090000e+09,True,A,96.0,99.0,NaN,...,Fixed Income,"U.S. - Corporate, Broad-based Investment Grad...",NaN,NaN,NaN,None,364,5.3453,6.07,1.628


In [12]:
etfs_lowrisk_new.drop(["Unnamed: 0", "simple_mean", "simple_median"], axis =1, inplace = True)

In [13]:
etfs_lowrisk_new.head()

,ticker,fund_name,one_yr_return,expense_ratio,aum,analyst_pick,grade,efficiency,tradability,fit,...,score_global_rank,sri_screening_criteria_exposure_%,carbon_intensity,main_segment,sub_segment,annual_to_monthly,roi,roi_7,roi_12,knn-imputer
0,TEGS,Trend Aggregation ESG ETF,NaN,1.43,3.645400e+05,True,NaN,48.0,NaN,NaN,...,NaN,NaN,NaN,Asset Allocation,Global - Target Outcome,NaN,NaN,NaN,None,-26.356
1,IWN,iShares Russell 2000 Value ETF,5.09,0.23,1.192000e+10,True,A,96.0,98.0,85.0,...,24.44,3.44,189.85,Equity,U.S. - Small Cap Value,0.005544,113971.278929,10661.728057,None,6.860
2,PAVE,Global X U.S. Infrastructure Development ETF,32.39,0.47,5.000000e+09,True,B,87.0,97.0,NaN,...,56.45,NaN,226.69,Equity,U.S. Infrastructure,NaN,NaN,NaN,None,11.706
3,FLOT,iShares Floating Rate Bond ETF,5.97,0.15,7.140000e+09,True,A,94.0,100.0,NaN,...,86.56,3.72,63.23,Fixed Income,"U.S. - Broad Market, Broad-based Investment G...",0.001283,45716.099348,8863.197425,None,1.550
4,IBDQ,iShares iBonds Dec 2025 Term Corporate ETF,1.11,0.10,2.090000e+09,True,A,96.0,99.0,NaN,...,62.25,8.25,187.22,Fixed Income,"U.S. - Corporate, Broad-based Investment Grad...",NaN,NaN,NaN,None,1.628


In [14]:
import psycopg2
# Import get_dataframe function from the sql module
from sql_functions import get_dataframe
from sql_functions import get_engine

In [15]:
table_name = 'etfs_lowrisk_new'

In [16]:
# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        etfs_lowrisk_new.to_sql(name=table_name, # Name of SQL table variable
                        con=engine, # Engine or connection
                        schema='public', # your class schema variable
                        if_exists='replace', # Drop the table before inserting new values 
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None
else:
    print('No engine')

The etfs_lowrisk_new table was imported successfully.
